<center><h1>That notebook is suited to run everything under google colab gpu runtime and sync files with google drive using pydrive</h1></center>

In [0]:
#In google colab /tmp is not mounted under tmpfs that results in slow shared memory speed
#Mount it only if running on google colab
!mount -t tmpfs tmpfs -o size=12G /tmp

In [0]:
!pip install PyDrive

In [0]:
def Init_PyDrive():
  global ID_BY_NAME
  from pydrive.auth import GoogleAuth
  from pydrive.drive import GoogleDrive
  from google.colab import auth
  from oauth2client.client import GoogleCredentials

  auth.authenticate_user()
  global gauth
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  global drive
  drive = GoogleDrive(gauth)
  ID_BY_NAME = dict()
  file_list = drive.ListFile({'q': "'root' in parents and trashed=false"}).GetList()
  for i in file_list:
    print(i['title'])
    ID_BY_NAME[i['title']] = i['id']

In [0]:
Init_PyDrive()

In [0]:
def Download_File(filename, savename):
  file = drive.CreateFile({'id': ID_BY_NAME[filename]})
  file.GetContentFile(savename)

In [0]:
def Upload_File(filename, drivename):
  file = drive.CreateFile({"title": drivename})
  file.SetContentFile(filename)
  file.Upload()

In [0]:
Download_File("WIDER_train.zip", "WIDER_train.zip")
Download_File("WIDER_val.zip", "WIDER_val.zip")
Download_File("wider_face_split.zip", "wider_face_split.zip")
Download_File("size_index_wider.txt", "size_index_wider.txt")

In [0]:
!unzip -q WIDER_train.zip
!unzip -q WIDER_val.zip
!unzip -q wider_face_split.zip

In [0]:
!rm -f *.zip

In [0]:
!ls -alh

In [0]:
Download_File("fdmobilenet.py", "fdmobilenet.py")

In [0]:
%matplotlib inline
import os
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
import tensorflow as tf
import math
from tensorflow.python.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Input, ZeroPadding2D, Reshape, UpSampling2D, Concatenate, Lambda, Layer, DepthwiseConv2D, Multiply, GlobalAveragePooling2D, Activation, Dropout, Add
from tensorflow.python.keras.layers.advanced_activations import LeakyReLU, PReLU
from tensorflow.python.keras.models import Model, load_model
from tensorflow.python.keras.optimizers import Adam
from tensorflow.python import keras
from tensorflow.python.keras import backend as K
import random
import time
import cv2
from PIL import Image
from multiprocessing import Process
from google.colab import files
from keras.utils import plot_model
from tensorflow.python.keras.callbacks import ModelCheckpoint, LearningRateScheduler
from imgaug import augmenters as ia

In [0]:
from tensorflow.python.keras.applications.mobilenet_v2 import MobileNetV2
from fdmobilenet import FDMobileNet

In [0]:
IMAGE_SIZE = 224
BATCH_SIZE = 64
SEED = int(time.time())
TRAIN_NUM_PARTS = 28
VAL_NUM_PARTS = 7
AVG_BYTES = 4 * 1000 * 1000
BOX_THRESHOLD = 0.5
NOFACES_RATE = 0.25
NOFACES_MIN_SIZE = IMAGE_SIZE
DEBUG = False

MIN_FACE_SIZE = 0.05
MIN_CROP_SIZE = IMAGE_SIZE
GRIDS = [IMAGE_SIZE // 32]

TRAIN_BBX_PATH = "./wider_face_split/wider_face_train_bbx_gt.txt"
TRAIN_IMAGES_PREFIX = "./WIDER_train/images/"
VAL_BBX_PATH = "./wider_face_split/wider_face_val_bbx_gt.txt"
VAL_IMAGES_PREFIX = "./WIDER_val/images/"
SIZE_INDEX = "./size_index_wider.txt"

HS_RANGE = 15
G_PROB = 0.5
G_MAX = 0.6
FLIP_PROB = 0.5
NOISE_LEVEL = 0.05
NOISE_PROB = 0.25
NOISE_CHANNEL_PROB = 0.5
CONTRAST_NORM_MIN = 0.75
CONTRAST_NORM_MAX = 1.5

def ATTRIBUTE_MASK(blur, expression, illumination, invalid, occlusion, pose):
  return occlusion != 2 and blur != 2

In [0]:
from multiprocessing import RawValue, RawArray
from numpy.ctypeslib import ctypes as ct
import numpy as np

class SharedImageArray:
  def __init__(self, maxmemory, maxlen):
    self.maxmemory = RawValue(ct.c_size_t, maxmemory)
    self.maxlen = RawValue(ct.c_size_t, maxlen)
    
    self.i = RawValue(ct.c_size_t, 0)
    self.i_offset = RawValue(ct.c_size_t, 0)
    
    self.shape_0 = RawArray(ct.c_size_t, maxlen)
    self.shape_1 = RawArray(ct.c_size_t, maxlen)
    self.offset = RawArray(ct.c_size_t, maxlen)
    self.size = RawArray(ct.c_size_t, maxlen)
    
    self.array = RawArray(ct.c_uint8, maxmemory)
  
  def push(self, arr):
    if self.i.value >= self.maxlen.value:
      raise ValueError('i >= maxlen')
    
    assert len(arr.shape) == 3
    assert arr.shape[2] == 3
    assert arr.dtype == np.uint8
    
    arr_size = arr.shape[0] * arr.shape[1] * 3
    
    if self.i_offset.value + arr_size >= self.maxmemory.value:
      raise ValueError('offset >= maxmemory')
    
    memoryview(self.array).cast('B')[self.i_offset.value:(self.i_offset.value + arr_size)] = arr.flatten()
    
    self.size[self.i.value] = arr_size
    self.offset[self.i.value] = self.i_offset.value
    self.shape_1[self.i.value] = arr.shape[1]
    self.shape_0[self.i.value] = arr.shape[0]

    self.i_offset.value += arr_size
    self.i.value += 1

  def len(self):
    return self.i.value
  
  def get(self, index):
    return np.frombuffer(memoryview(self.array).cast('B'), np.uint8, self.size[index], self.offset[index]).reshape(self.shape_0[index], self.shape_1[index], 3)
  
  def clear(self):
    self.i.value = 0
    self.i_offset.value = 0

In [0]:
class DataPipeline(object):
  def __random_quad(self, x1, y1, x2, y2, W, H):
    if x1 > W or x2 > W or y1 > H or y2 > H:
      return (None, None), (None, None)
   
    VERTICAL = 0
    HORIZONTAL = 1

    def intersect(p, k, l, r, t):
        if t == VERTICAL:
            x_i = p
            y_i = p - k
            if y_i >= l and y_i <= r:
                return (x_i, y_i)
            return None
        else:
            y_i = p
            x_i = p + k
            if x_i >= l and x_i <= r:
                return (x_i, y_i)
            return None

    def cuts(k):
        m1 = []
        m2 = []

        a1 = intersect(0, k, 0, y1, VERTICAL)
        a2 = intersect(x1, k, 0, y1, VERTICAL)
        a3 = intersect(0, k, 0, x1, HORIZONTAL)
        a4 = intersect(y1, k, 0, x1, HORIZONTAL)
        a5 = intersect(x2, k, y2, H, VERTICAL)
        a6 = intersect(W, k, y2, H, VERTICAL)
        a7 = intersect(y2, k, x2, W, HORIZONTAL)
        a8 = intersect(H, k, x2, W, HORIZONTAL)

        for i in [a1, a2, a3, a4]:
            if not(i is None):
                m1.append(i)

        for i in [a5, a6, a7, a8]:
            if not(i is None):
                m2.append(i)

        return m1, m2

    def d(a, b):
        return math.sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2)

    def randomk():
        def function_fix(a, b):
          assert b >= a and a >= 0 and b >= 0 and a <= 1 and b <= 1
          if self.function_fix_approx is None:
            return b - a
          return self.function_fix_approx[int(round(b * (len(self.function_fix_approx) - 1)))] - self.function_fix_approx[int(round(a * (len(self.function_fix_approx) - 1)))]
      
        K = 32
        
        x = np.arange(-y1, x1, (x1 + y1) / K)
        y = [0] * len(x)
        s = 0
        
        for i in range(1, len(x)):
            l = x[i] - x[i - 1]
            p = (x[i] + x[i - 1]) / 2
            m1, m2 = cuts(p)
            if len(m1) < 2 or len(m2) < 2:
              y[i] = 0
            else:
              dx = x2 - x1
              dm1x1 = x1 - max(m1[0][0], m1[1][0])
              dm1x2 = x1 - min(m1[0][0], m1[1][0])
              dm2x1 = min(m2[0][0], m2[1][0]) - x2
              dm2x2 = max(m2[0][0], m2[1][0]) - x2
              y[i] = function_fix(dx / (dx + dm1x2), dx / (dx + dm1x1)) * function_fix(dx / (dx + dm2x2), dx / (dx + dm2x1)) * l      
            s += y[i]

        r = random.uniform(0, s)

        for i in range(1, len(x)):
            if r - y[i] <= 0:
                return random.uniform(x[i - 1], x[i])
            r -= y[i]

        return random.uniform(x[-2], x[-1])

    def v(a, b):
        return (b[0] - a[0], b[1] - a[1])

    def m(a, b):
        return (a[0] * b, a[1] * b)

    def s(a, b):
        return (b[0] + a[0], b[1] + a[1])
      
    def randomr(l, r):
      assert l >= 0 and l <= 1 and r >= 0 and r <= 1 and l <= r
      
      if self.function_fix_approx is None:
        return random.uniform(l, r)
      
      l = int(math.ceil(l * (len(self.function_fix_approx) - 1)))
      r = int(math.floor(r * (len(self.function_fix_approx) - 1)))
      
      if l >= r:
        return l
      
      x = random.uniform(self.function_fix_approx[l], self.function_fix_approx[r])
      
      i = int(round(x * (len(self.inverse_function_fix_approx) - 1)))
      
      return self.inverse_function_fix_approx[i]
    
    if x1 + y1 == 0:
      return (None, None), (None, None)
    
    k = randomk()
    m1, m2 = cuts(k)
    
    if len(m1) == 0 or len(m2) == 0:
      return (None, None), (None, None)
    
    if m1[0][0] > m1[1][0]:
      m1[0], m1[1] = m1[1], m1[0]
    
    if m2[0][0] > m2[1][0]:
      m2[0], m2[1] = m2[1], m2[0]
    
    if len(m1) == 1:
      v1 = (0, 0)
    else:
      v1 = v(m1[1], m1[0])
    if len(m2) == 1:
      v2 = (0, 0)
    else:
      v2 = v(m2[0], m2[1])

    l1, l2 = d(m1[0], m1[1]), d(m2[0], m2[1])
    
    dx = x2 - x1
    dm1x1 = x1 - max(m1[0][0], m1[1][0])
    dm1x2 = x1 - min(m1[0][0], m1[1][0])
    dm2x1 = min(m2[0][0], m2[1][0]) - x2
    dm2x2 = max(m2[0][0], m2[1][0]) - x2
    
    if dm1x2 - dm1x1 == 0:
      r1 = 0
    else:
      r1 = randomr(dx / (dx + dm1x2), dx / (dx + dm1x1))
      r1 = dx * (1 - r1) / r1
      r1 -= dm1x1
      r1 /= dm1x2 - dm1x1
      
    if dm2x2 - dm2x1 == 0:
      r2 = 0
    else:
      r2 = randomr(dx / (dx + dm2x2), dx / (dx + dm2x1))
      r2 = dx * (1 - r2) / r2
      r2 -= dm2x1
      r2 /= dm2x2 - dm2x1

    ans = (s(m1[1], m(v1, r1))), (s(m2[0], m(v2, r2)))
    if ans[0][0] > ans[1][0]:
        return (ans[1], ans[0])

    return ans
  
  def __random_outside(self, W, H, x1, y1, x2, y2, min_size):
    def random_quad_inside_rectangle(x1, y1, x2, y2):
      maxd = min(x2 - x1, y2 - y1)
      d = random.uniform(min_size, maxd)
      shiftx = random.uniform(0, x2 - x1 - d)
      shifty = random.uniform(0, y2 - y1 - d)

      return (x1 + shiftx, y1 + shifty), (x1 + shiftx + d, y1 + shifty + d)

    if min(H, x1) >= min_size:
      Spl = H * x1
    else:
      Spl = 0
    if min(H, W - x2) >= min_size:
      Spr = H * (W - x2)
    else:
      Spr = 0
    if min(W, H - y2) >= min_size:
      Spu = W * (H - y2)
    else:
      Spu = 0
    if min(W, y1) >= min_size:
      Spd = W * y1
    else:
      Spd = 0
    S = Spl + Spr + Spu + Spd

    if S < 1e-9:
      return (None, None), (None, None)

    r = random.uniform(0, S)
    if r <= Spl:
      return random_quad_inside_rectangle(0, 0, x1, H)
    elif r <= Spl + Spr:
      return random_quad_inside_rectangle(x2, 0, W, H)
    elif r <= Spl + Spr + Spu:
      return random_quad_inside_rectangle(0, y2, W, H)
    else:
      return random_quad_inside_rectangle(0, 0, W, y1)
  
  @staticmethod
  def __update_part(block_idx, part_size, config_data, parts_list):
    j = block_idx * part_size
    for i in range(part_size):
      parts_list.push(np.asarray(Image.open(config_data[j][0]), dtype=np.uint8))
      j += 1
      if j == len(config_data):
        j = 0
  
  def __parts_data_index(self, current_block, i):
    j = current_block * self.part_size + i
    if j >= len(self.config_data):
      j-= len(self.config_data)
    return j
  
  def __init__(self, config, images_prefix, image_size, batch_size, seed, parts_cnt, avg_bytes, grids, min_crop_size, attribute_mask, function_fix_approx=None, inverse_function_fix_approx=None, box_threshold=0.5, debug=False, min_face_size=2, nofaces_rate=0, nofaces_min_size=2, hs_range=15, g_prob=0.5, g_max=0.6, flip_prob=0.5, noise_level=0.05, noise_prob=0.5, noise_channel_prob=0.5, contrast_norm_min=0.75, contrast_norm_max=1.5, boxes_only = False, size_index=None):
    assert parts_cnt > 1
    
    self.config_data = []
    
    max_boxes = 0
    cf = open(config, "r")
    while True:
      take_image = True
      image = cf.readline()[:-1]
      if len(image) <= 1:
        break
      config_item = [images_prefix + "/" + image, [], ""]
      n = int(cf.readline())
      for i in range(n):
        x1, y1, w, h, blur, expression, illumination, invalid, occlusion, pose = list(map(int, cf.readline().split()))
        if x1 >= 0 and y1 >= 0 and w >= 2 and h >= 2:
          config_item[1].append((x1, y1, w, h, not(attribute_mask(blur, expression, illumination, invalid, occlusion, pose))))
        else:
          take_image = False
      if len(config_item[1]) > 0 and take_image:
        max_boxes = max(max_boxes, len(config_item[1]))
        config_item[2] = image
        self.config_data.append(config_item)
    cf.close()
    
    self.debug = debug
    
    if self.debug:
      print("DEBUG: metadata loaded, %d images total, %d max boxes" % (len(self.config_data), max_boxes))
    
    random.seed(seed)
    np.random.seed(seed)
    np.random.shuffle(self.config_data)
    self.min_face_size = min_face_size
    self.boxes_only = boxes_only
    self.nofaces_rate = nofaces_rate
    self.nofaces_min_size = nofaces_min_size
    self.conflicts = 0
    self.grids = grids
    self.function_fix_approx = function_fix_approx
    self.inverse_function_fix_approx = inverse_function_fix_approx
    self.skipped = 0
    self.image_size = image_size
    self.box_threshold = box_threshold
    self.i = 0
    self.batch_size = batch_size
    self.part_size = len(self.config_data) // parts_cnt
    self.parts_cnt = parts_cnt
    if not(self.boxes_only):
      self.parts = [SharedImageArray(avg_bytes * self.part_size, self.part_size), SharedImageArray(avg_bytes * self.part_size, self.part_size)]
    self.hs_range = hs_range
    self.g_prob = g_prob
    self.g_max = g_max
    self.flip_prob = flip_prob
    self.noise_level = noise_level
    self.noise_prob = noise_prob
    self.noise_channel_prob = noise_channel_prob
    self.contrast_norm_min = contrast_norm_min
    self.contrast_norm_max = contrast_norm_max
    self.min_crop_size = min_crop_size
    if size_index is None:
      self.size_index = None
    else:
      self.size_index = dict()
      sif = open(size_index, "r")
      for line in sif.readlines():
        fname, w, h = line[:-1].split()
        w = int(w)
        h = int(h)
        self.size_index[fname] = (w, h)
    
    if self.debug:
      print("DEBUG: loading initial data")
    if not(self.boxes_only):
      self.__update_part(0, self.part_size, self.config_data, self.parts[0])
    if self.debug:
      print("DEBUG: initial data loaded")
    
    self.current_part = 0
    self.current_block = 0
    self.block_idx = 1
    self.part_status = [1, 0]
    self.color_augmentations = ia.Sequential([
      ia.AddToHueAndSaturation((-self.hs_range, self.hs_range)),
      ia.Sometimes(self.g_prob, ia.Grayscale((0, self.g_max))),
      ia.ContrastNormalization((self.contrast_norm_min, self.contrast_norm_max)),
      ia.Sometimes(self.noise_prob, ia.AdditiveGaussianNoise(loc=0, scale=(0.0, self.noise_level * 255), per_channel=self.noise_channel_prob))
    ])
   
  def __intersect_sections(self, ax1, ax2, bx1, bx2):
    if ax1 > bx1:
      ax1, bx1 = bx1, ax1
      ax2, bx2 = bx2, ax2
    
    if ax2 <= bx1:
      return None, None
    
    return bx1, min(ax2, bx2)
  
  def __intersect_boxes(self, ax1, ay1, ax2, ay2, bx1, by1, bx2, by2):
    assert ax2 >= ax1 and ay2 >= ay1 and bx2 >= bx1 and by2 >= by1
    
    sa = (ax2 - ax1) * (ay2 - ay1)
    sb = (bx2 - bx1) * (by2 - by1)
    
    if sa == 0 or sb == 0:
      return None, None, None, None, None
    
    ix1, ix2 = self.__intersect_sections(ax1, ax2, bx1, bx2)
    iy1, iy2 = self.__intersect_sections(ay1, ay2, by1, by2)
    
    if ix1 is None or iy1 is None:
      return None, None, None, None, None
    
    si = (ix2 - ix1) * (iy2 - iy1)
    
    if si == 0:
      return None, None, None, None, None
    
    relsize = si / min(sa, sb)
    
    return ix1, iy1, ix2, iy2, relsize
  
  def color_augmentations(self, image_batch):
    return self.color_augmentations.augment_images(image_batch)
  
  def flow(self):
    while True:
      if self.part_status[1 - self.current_part] == 0 and not(self.boxes_only):
        if self.debug:
          print("DEBUG: loading additional data")
        self.parts[1 - self.current_part].clear()
        self.part_loader = Process(target=self.__update_part, args=(self.block_idx, self.part_size, self.config_data, self.parts[1 - self.current_part]))
        self.part_loader.start()
        self.part_status[1 - self.current_part] = -1
        
      if not(self.boxes_only) and not(self.part_loader.is_alive()) and self.part_status[1 - self.current_part] == -1:
        if self.debug:
          print("DEBUG: additional data loaded")
        self.part_status[1 - self.current_part] = 1
      
      if self.boxes_only:
        self.part_status[1 - self.current_part] = 1
      
      if self.part_status[self.current_part] == 2 and self.part_status[1 - self.current_part] == 1:
        if self.debug:
          print("DEBUG: parts flip flop")
        self.current_part = 1 - self.current_part
        self.part_status[1 - self.current_part] = 0
        
        self.i = 0
        self.current_block = self.block_idx
        self.block_idx += 1
        if self.block_idx == self.parts_cnt:
          self.block_idx = 0
        continue
      
      if not(self.boxes_only):
        x_yield = np.zeros((self.batch_size, self.image_size, self.image_size, 3), dtype=np.uint8)
      y_yield = []
      if not(self.boxes_only):
        for i in range(len(self.grids)):
          y_yield.append(np.zeros((self.batch_size, self.grids[i], self.grids[i], 6), dtype=np.float))
      
      for k in range(self.batch_size):
        while True:
          if not(self.boxes_only):
            image = self.parts[self.current_part].get(self.i)
            H = image.shape[0]
            W = image.shape[1]
          else:
            image_name = self.config_data[self.__parts_data_index(self.current_block, self.i)][2]
            W, H = self.size_index[image_name]
          boxes = self.config_data[self.__parts_data_index(self.current_block, self.i)][1]
          if random.uniform(0, 1) < self.nofaces_rate:
            x1, y1, w, h, hard = boxes[0]
            y1 = (H - 1) - y1 - h
            x2 = x1 + w
            y2 = y1 + h
            for i in range(1, len(boxes)):
              cx1, cy1, cw, ch, hard = boxes[i]
              cy1 = (H - 1) - cy1 - ch
              cx2 = cx1 + cw
              cy2 = cy1 + ch
              
              y1 = min(y1, cy1)
              y2 = max(y2, cy2)
              x1 = min(x1, cx1)
              x2 = max(x2, cx2)
              
            (qx1, qy1), (qx2, qy2) = self.__random_outside(W - 1, H - 1, x1, y1, x2, y2, self.nofaces_min_size)
            if qx1 is None:
              self.skipped += 1
              self.i += 1
              if self.i == self.part_size:
                self.i = 0
                self.part_status[self.current_part] = 2
            else:
              break
              
          else:
            not_hard_indexes = []
            for i in range(len(boxes)):
              if not(boxes[i][4]):
                not_hard_indexes.append(i)
            if len(not_hard_indexes) == 0:
              self.skipped += 1
              self.i += 1
              if self.i == self.part_size:
                self.i = 0
                self.part_status[self.current_part] = 2
              continue
            
            keep_face = random.randint(0, len(not_hard_indexes) - 1)
            x1, y1, w, h, hard = boxes[not_hard_indexes[keep_face]]
            y1 = (H - 1) - y1 - h
            x2 = x1 + w
            y2 = y1 + h
            (qx1, qy1), (qx2, qy2) = self.__random_quad(x1, y1, x2, y2, W - 1, H - 1)
            if qx1 is None or qx1 < 0 or qx2 < 0 or qy1 < 0 or qy2 < 0 or qx1 > W - 1 or qx2 > W - 1 or qy1 > H - 1 or qy2 > H - 1 or qx2 - qx1 < self.min_crop_size or qy2 - qy1 < self.min_crop_size:
              self.skipped += 1
              self.i += 1
              if self.i == self.part_size:
                self.i = 0
                self.part_status[self.current_part] = 2
            else:
              break
            
        qy1 = (H - 1) - int(qy1)
        qx1 = int(qx1)
        qy2 = (H - 1) - int(qy2)
        qx2 = int(qx2)
        qy1, qy2 = qy2, qy1
        
        if qx2 - qx1 == qy2 - qy1 + 1:
          qx1 += 1
        elif qy2 - qy1 == qx2 - qx1 + 1:
          qy1 += 1
        
        if not(self.boxes_only):
          x_yield[k] = cv2.resize(image[qy1:(qy2 + 1), qx1:(qx2 + 1)], (self.image_size, self.image_size))
        
        if random.uniform(0, 1) <= self.flip_prob:
          fliplr = True
          if not(self.boxes_only):
            x_yield[k] = np.fliplr(x_yield[k])
        else:
          fliplr = False
        
        downsample = 1 / (qy2 - qy1 + 1)
        
        if not(self.boxes_only):
          for box_x1, box_y1, box_w, box_h, hard in boxes:
            ibox_x1, ibox_y1, ibox_x2, ibox_y2, relsize = self.__intersect_boxes(box_x1, box_y1, box_x1 + box_w, box_y1 + box_h, qx1, qy1, qx2, qy2)
            if not(ibox_x1 is None):
              if relsize < self.box_threshold:
                hard = True
              ibox = [(ibox_x1 - qx1) * downsample, (ibox_y1 - qy1) * downsample, (ibox_x2 - qx1) * downsample, (ibox_y2 - qy1) * downsample]
              if fliplr:
                ibox[0] = 1 - ibox[0]
                ibox[2] = 1 - ibox[2]
                ibox[0], ibox[2] = ibox[2], ibox[0]
              wx = ibox[2] - ibox[0]
              wy = ibox[3] - ibox[1]
              cx = ibox[0] + wx / 2
              cy = ibox[1] + wy / 2
              
              if wx < self.min_face_size or wy < self.min_face_size:
                hard = True

              for grid in range(len(self.grids)):
                grid_i = min(int(cx * self.grids[grid]), self.grids[grid] - 1)
                grid_j = min(int(cy * self.grids[grid]), self.grids[grid] - 1)
                sx = (cx - grid_i / self.grids[grid]) * self.grids[grid]
                sy = (cy - grid_j / self.grids[grid]) * self.grids[grid]

                if y_yield[grid][k][grid_i][grid_j][4] == 1:
                    self.conflicts += 1

                if not(hard) and (y_yield[grid][k][grid_i][grid_j][4] == 0 or y_yield[grid][k][grid_i][grid_j][2] * y_yield[grid][k][grid_i][grid_j][3] < wx * self.grids[grid] * wy * self.grids[grid]):
                  y_yield[grid][k][grid_i][grid_j] = (sx,  sy,  wx * self.grids[grid],  wy * self.grids[grid], 1, 0)
                  
                if hard and y_yield[grid][k][grid_i][grid_j][4] == 0 and (y_yield[grid][k][grid_i][grid_j][5] == 0 or y_yield[grid][k][grid_i][grid_j][2] * y_yield[grid][k][grid_i][grid_j][3] < wx * self.grids[grid] * wy * self.grids[grid]):
                  y_yield[grid][k][grid_i][grid_j] = (sx,  sy,  wx * self.grids[grid],  wy * self.grids[grid], 0, 1)
        else:
          c_yield = []
          for box_x1, box_y1, box_w, box_h, hard in boxes:
            ibox_x1, ibox_y1, ibox_x2, ibox_y2, relsize = self.__intersect_boxes(box_x1, box_y1, box_x1 + box_w, box_y1 + box_h, qx1, qy1, qx2, qy2)
            if not(ibox_x1 is None):
              if relsize < self.box_threshold:
                hard = True
              ibox = [(ibox_x1 - qx1) * downsample, (ibox_y1 - qy1) * downsample, (ibox_x2 - qx1) * downsample, (ibox_y2 - qy1) * downsample]
              if fliplr:
                ibox[0] = 1 - ibox[0]
                ibox[2] = 1 - ibox[2]
                ibox[0], ibox[2] = ibox[2], ibox[0]
              wx = ibox[2] - ibox[0]
              wy = ibox[3] - ibox[1]
              
              if wx < self.min_face_size or wy < self.min_face_size:
                  hard = True
                  
              if not(hard):
                c_yield.append(ibox)
          y_yield.append(c_yield)
        
        
        self.i += 1
        if self.i == self.part_size:
          self.i = 0
          self.part_status[self.current_part] = 2
          
      if self.debug:
        print("DEBUG: %d images skipped, %d conflicts" % (self.skipped, self.conflicts))
      self.skipped = 0
      self.conflicts = 0
      if self.boxes_only:
        yield(y_yield)
      else:
        yield((self.color_augmentations.augment_images(x_yield), y_yield))

In [0]:
def distribution_histogram(function_fix, inverse_functiom_fix, num_batches, num_bins=20):
  boxes = []
  generator = DataPipeline(TRAIN_BBX_PATH, TRAIN_IMAGES_PREFIX, IMAGE_SIZE, BATCH_SIZE, SEED, TRAIN_NUM_PARTS, AVG_BYTES, GRIDS, MIN_CROP_SIZE, ATTRIBUTE_MASK, function_fix, inverse_functiom_fix, BOX_THRESHOLD, DEBUG, MIN_FACE_SIZE, NOFACES_RATE, NOFACES_MIN_SIZE, HS_RANGE, G_PROB, G_MAX, FLIP_PROB, NOISE_LEVEL, NOISE_PROB, NOISE_CHANNEL_PROB, CONTRAST_NORM_MIN, CONTRAST_NORM_MAX, True, SIZE_INDEX)
  flow = generator.flow()
  for i in range(num_batches):
    batch = next(flow)
    for image in batch:
      for box in image:
        boxes.append((box[2] - box[0], box[3] - box[1]))
  boxes = np.array(boxes)
  histogram = np.array(np.histogram(np.array(boxes[:, 1], dtype=np.float32), bins=num_bins, range=(0, 1))[0], dtype=np.float32)
  histogram /= np.mean(histogram)
  
  return histogram

def distribution_loss(function_fix, inverse_functiom_fix, num_batches, num_bins=20):
  return np.std(distribution_histogram(function_fix, inverse_functiom_fix, num_batches, num_bins))

def inverse_distribution_function(f, k):
  inv = np.zeros((k,))
  for i in range(len(f)):
    inv[int(math.ceil(f[i] * (k - 1)))] = i / (len(f) - 1)
    inv[int(math.floor(f[i] * (k - 1)))] = i / (len(f) - 1)
  inv[0] = 0
  inv[k - 1] = 1
  for i in range(k):
    if i != 0 and i != k - 1 and inv[i] == 0:
      if inv[i - 1] != 0 and inv[i + 1] != 0:
        inv[i] = inv[i - 1] / 2 + inv[i + 1] / 2
      else:
        inv[i] = inv[i - 1] + inv[i + 1]
    
  return inv

def density_to_function(d):
  f = np.zeros((len(d) + 1,))
  f[0] = 0
  f[1] = d[0] / len(d)
  for i in range(1, len(d)):
    f[i + 1] = f[i] + d[i] / len(d)
  f[len(d)] = 1
  return f

def function_space(p, k=100000):
  f = np.linspace(0, 1, k) ** p[0] + p[1]
  f /= np.mean(f)
  return f

def f(p, num_batches=300, num_bins=20):
  d = function_space(p)
  f = density_to_function(d)
  invf = inverse_distribution_function(f, len(f))
  return distribution_loss(f, invf, num_batches, num_bins)

def f_hist(p, num_batches=300, num_bins=20):
  d = function_space(p)
  f = density_to_function(d)
  invf = inverse_distribution_function(f, len(f))
  return distribution_histogram(f, invf, num_batches, num_bins)

def gridsearch(pspace, cspace, num_batches=1000):
  best_val = 1e9
  best_p = -1
  best_c = -1
  heatmap = np.zeros((len(pspace), len(cspace),), dtype=np.float32)
  for i in range(len(pspace)):
    for j in range(len(cspace)):
      p = pspace[i]
      c = cspace[j]
      val = f([p, c], num_batches)
      if val < best_val:
        best_val = val
        best_p = p
        best_c = c
      print(p, c, val)
      heatmap[i][j] = val
  plt.figure(figsize=(10, 10))
  plt.contourf(pspace, cspace, heatmap, cmap='jet')
  plt.show()
  
  return (best_p, best_c, best_val)

In [0]:
#gridsearch(np.linspace(0, 3, 10), np.linspace(0, 1, 10), 64000 // BATCH_SIZE)

In [0]:
P = 0
C = 1

plt.plot(f_hist([P, C], 64000 // BATCH_SIZE, 100))
DENSITY_FIX_APPROX = function_space([P, C])
FUNCTION_FIX_APPROX = density_to_function(DENSITY_FIX_APPROX)
INVERSE_FUNCTION_FIX_APPROX = inverse_distribution_function(FUNCTION_FIX_APPROX, len(FUNCTION_FIX_APPROX))
plt.figure(figsize=(10, 10))
plt.plot(FUNCTION_FIX_APPROX)
plt.plot(INVERSE_FUNCTION_FIX_APPROX)
plt.show()
plt.figure(figsize=(10, 10))
plt.plot(DENSITY_FIX_APPROX)
plt.show()

In [0]:
train_generator = DataPipeline(TRAIN_BBX_PATH, TRAIN_IMAGES_PREFIX, IMAGE_SIZE, BATCH_SIZE, SEED, TRAIN_NUM_PARTS, AVG_BYTES, GRIDS, MIN_CROP_SIZE, ATTRIBUTE_MASK, FUNCTION_FIX_APPROX, INVERSE_FUNCTION_FIX_APPROX, BOX_THRESHOLD, DEBUG, MIN_FACE_SIZE, NOFACES_RATE, NOFACES_MIN_SIZE, HS_RANGE, G_PROB, G_MAX, FLIP_PROB, NOISE_LEVEL, NOISE_PROB, NOISE_CHANNEL_PROB, CONTRAST_NORM_MIN, CONTRAST_NORM_MAX, False, None)
train_flow = train_generator.flow()

In [0]:
val_generator = DataPipeline(VAL_BBX_PATH, VAL_IMAGES_PREFIX, IMAGE_SIZE, BATCH_SIZE, SEED, VAL_NUM_PARTS, AVG_BYTES, GRIDS, MIN_CROP_SIZE, ATTRIBUTE_MASK, FUNCTION_FIX_APPROX, INVERSE_FUNCTION_FIX_APPROX, BOX_THRESHOLD, DEBUG, MIN_FACE_SIZE, NOFACES_RATE, NOFACES_MIN_SIZE, HS_RANGE, G_PROB, G_MAX, FLIP_PROB, NOISE_LEVEL, NOISE_PROB, NOISE_CHANNEL_PROB, CONTRAST_NORM_MIN, CONTRAST_NORM_MAX, False, None)
val_flow = val_generator.flow()

In [0]:
def visualise_batch_with_boxes(batch):
  for i in range(BATCH_SIZE):
    plt.figure(figsize=(5, 5))
    plt.imshow(batch[0][i])
    for g in range(len(GRIDS)):
      for x in range(GRIDS[g]):
        for y in range(GRIDS[g]):
          p = batch[1][g][i][x][y][4]
          hard = batch[1][g][i][x][y][5]
          if p == 1 or hard == 1:
            cx = x + batch[1][g][i][x][y][0]
            cy = y + batch[1][g][i][x][y][1]
            wx = batch[1][g][i][x][y][2]
            wy = batch[1][g][i][x][y][3]
            lx = cx - wx / 2
            ly = cy - wy / 2
            rx = cx + wx / 2
            ry = cy + wy / 2
            lx = int(round(lx * IMAGE_SIZE / GRIDS[g]))
            rx = int(round(rx * IMAGE_SIZE / GRIDS[g]))
            ly = int(round(ly * IMAGE_SIZE / GRIDS[g]))
            ry = int(round(ry * IMAGE_SIZE / GRIDS[g]))
            if hard == 0:
              plt.gca().add_patch(Rectangle((lx,ly),rx - lx,ry - ly,linewidth=2,edgecolor='g',facecolor='none'))
            else:
              plt.gca().add_patch(Rectangle((lx,ly),rx - lx,ry - ly,linewidth=2,edgecolor='r',facecolor='none'))
    plt.show()

def measure_batch_time(flow):
  start = time.time()
  batch = next(flow)
  print("Batch time: " + str(time.time() - start))
  return batch

#visualise_batch_with_boxes(measure_batch_time(train_flow))

In [0]:
LAMBDA_BOXES = 1
LAMBDA_CONFIDENCE = 3/4
LAMBDA_FP = 1/4
EPS = 1e-9
ALPHA = 0.1
IOU_THRESHOLD = 0.8

def yolo_like_loss_exp(y_true, y_pred):
  grid_size = int(y_pred.shape.as_list()[1])
  
  tcx     = tf.reshape(y_true[..., 0], (-1, grid_size ** 2))
  tcy     = tf.reshape(y_true[..., 1], (-1, grid_size ** 2))
  twx     = tf.reshape(y_true[..., 2], (-1, grid_size ** 2))
  twy     = tf.reshape(y_true[..., 3], (-1, grid_size ** 2))
  exists  = tf.reshape(y_true[..., 4], (-1, grid_size ** 2))
  #hard  = tf.reshape(y_true[..., 5], (-1, grid_size ** 2))
  
  pcx = tf.sigmoid(tf.reshape(y_pred[..., 0], (-1, grid_size ** 2)))
  pcy = tf.sigmoid(tf.reshape(y_pred[..., 1], (-1, grid_size ** 2)))
  y_pred_2 = tf.reshape(y_pred[..., 2], (-1, grid_size ** 2))
  pwx = tf.minimum(tf.exp(y_pred_2 / grid_size), grid_size) + tf.maximum(ALPHA * (y_pred_2 - grid_size * math.log(grid_size)), 0)
  y_pred_3 = tf.reshape(y_pred[..., 3], (-1, grid_size ** 2))
  pwy = tf.minimum(tf.exp(y_pred_3 / grid_size), grid_size) + tf.maximum(ALPHA * (y_pred_3 - grid_size * math.log(grid_size)), 0)
  p   = tf.sigmoid(tf.reshape(y_pred[..., 4], (-1, grid_size ** 2)))
  
  loss_boxes = tf.reduce_sum(exists * ((tcx - pcx) ** 2 + (tcy - pcy) ** 2 + (tf.sqrt(twx) - tf.sqrt(pwx)) ** 2 + (tf.sqrt(twy) - tf.sqrt(pwy)) ** 2)) / (tf.reduce_sum(exists) + EPS)
  
  tx1 = tcx - twx / 2
  tx2 = tcx + twx / 2
  ty1 = tcy - twy / 2
  ty2 = tcy + twy / 2
  
  px1 = pcx - pwx / 2
  px2 = pcx + pwx / 2
  py1 = pcy - pwy / 2
  py2 = pcy + pwy / 2
  
  tx1p = tf.reshape(tf.tile(tx1, (1, grid_size ** 2)), (-1, grid_size ** 2, grid_size ** 2))
  tx2p = tf.reshape(tf.tile(tx2, (1, grid_size ** 2)), (-1, grid_size ** 2, grid_size ** 2))
  ty1p = tf.reshape(tf.tile(ty1, (1, grid_size ** 2)), (-1, grid_size ** 2, grid_size ** 2))
  ty2p = tf.reshape(tf.tile(ty2, (1, grid_size ** 2)), (-1, grid_size ** 2, grid_size ** 2))
  
  px1p = tf.transpose(tf.reshape(tf.tile(px1, (1, grid_size ** 2)), (-1, grid_size ** 2, grid_size ** 2)), (0, 2, 1))
  px2p = tf.transpose(tf.reshape(tf.tile(px2, (1, grid_size ** 2)), (-1, grid_size ** 2, grid_size ** 2)), (0, 2, 1))
  py1p = tf.transpose(tf.reshape(tf.tile(py1, (1, grid_size ** 2)), (-1, grid_size ** 2, grid_size ** 2)), (0, 2, 1))
  py2p = tf.transpose(tf.reshape(tf.tile(py2, (1, grid_size ** 2)), (-1, grid_size ** 2, grid_size ** 2)), (0, 2, 1))
  
  twxp = tf.reshape(tf.tile(twx, (1, grid_size ** 2)), (-1, grid_size ** 2, grid_size ** 2))
  twyp = tf.reshape(tf.tile(twy, (1, grid_size ** 2)), (-1, grid_size ** 2, grid_size ** 2))
  
  pwxp = tf.transpose(tf.reshape(tf.tile(pwx, (1, grid_size ** 2)), (-1, grid_size ** 2, grid_size ** 2)), (0, 2, 1))
  pwyp = tf.transpose(tf.reshape(tf.tile(pwy, (1, grid_size ** 2)), (-1, grid_size ** 2, grid_size ** 2)), (0, 2, 1))
  
  intersectionp = tf.maximum(tf.minimum(tx2p, px2p) - tf.maximum(tx1p, px1p), 0) * tf.maximum(tf.minimum(ty2p, py2p) - tf.maximum(ty1p, py1p), 0)
  ioup = intersectionp / (twxp * twyp + pwxp * pwyp - intersectionp + EPS)
  
  fp_mask = tf.cast(tf.reduce_max(ioup, axis=2) < IOU_THRESHOLD, tf.float32)
  
  intersection = tf.maximum(tf.minimum(tx2, px2) - tf.maximum(tx1, px1), 0) * tf.maximum(tf.minimum(ty2, py2) - tf.maximum(ty1, py1), 0)
  iou = intersection / (twx * twy + pwx * pwy - intersection + EPS)
  
  loss_confidence = tf.reduce_sum(exists * (iou - p) ** 2) / (tf.reduce_sum((1 - exists) * fp_mask) + tf.reduce_sum(exists) + EPS)
  
  loss_fp = tf.reduce_sum((1 - exists) * fp_mask * p ** 2) / (tf.reduce_sum((1 - exists) * fp_mask) + tf.reduce_sum(exists) + EPS)
  
  return LAMBDA_BOXES * loss_boxes + LAMBDA_CONFIDENCE * loss_confidence + LAMBDA_FP * loss_fp

In [0]:
#tf.keras.backend.clear_session()

In [0]:
input_tensor = Input(shape=(IMAGE_SIZE, IMAGE_SIZE, 3))
output_tensor = MobileNetV2(input_tensor=input_tensor, alpha=0.75, weights=None, include_top=False).output
output_tensor = ZeroPadding2D()(output_tensor)
output_tensor = Conv2D(kernel_size=(3, 3), filters=5)(output_tensor)

model = Model(inputs=input_tensor, outputs=output_tensor)

In [0]:
model.summary()

In [0]:
#model.load_weights("1.h5")

In [0]:
STEPS = 1024
EPOCHS = 30
VAL_STEPS = STEPS // 4
INITIAL_EPOCH = 0

model.compile(optimizer=Adam(lr=1e-3), loss=yolo_like_loss_exp)

checkpointer = ModelCheckpoint("tmp19.h5", monitor='val_loss', verbose=1, save_best_only=True, save_weights_only=True)
lr_scheldure = LearningRateScheduler(lambda epoch, lr: 0.9 ** epoch * 1e-3, verbose=1)

model.fit_generator(
                    train_flow,
                    steps_per_epoch=STEPS,
                    epochs=EPOCHS,
                    verbose=1,
                    use_multiprocessing=False,
                    workers=1,
                    validation_data=val_flow,
                    validation_steps=VAL_STEPS,
                    initial_epoch=INITIAL_EPOCH,
                    callbacks=[checkpointer, lr_scheldure]
                   )

In [0]:
model.save_weights("new5.h5")

In [0]:
Init_PyDrive()
Upload_File("tmp19.h5", "tmp19.h5")

In [0]:
Init_PyDrive()
Download_File("tmp18.h5", "1.h5")